In [6]:
import pandas as pd
import glob, os

path = r'./topics'

all_files = glob.glob(os.path.join(path, '*.data'))
filename_list = []
opinion_text = []

for file_ in all_files:
    df = pd.read_table(file_, index_col=None, header=0, encoding='latin1')
    filename_ = file_.split('/')[-1]
    filename = filename_.split('.')[0]

    filename_list.append(filename)
    opinion_text.append(df.to_string())

document_df = pd.DataFrame({ 'filename': filename_list, 'opinion_text': opinion_text })
document_df.head()

,filename,opinion_text
0,battery-life_ipod_nano_8gb,...
1,gas_mileage_toyota_camry_2007,...
2,room_holiday_inn_london,...
3,location_holiday_inn_london,...
4,staff_bestwestern_hotel_sfo,...


In [7]:
from nltk.stem import WordNetLemmatizer
import nltk
import string

remove_punct_dict = dict((ord(punct), None) for punct in string.punctuation)
lemmar = WordNetLemmatizer()

def LemTokens(tokens):
    return [lemmar.lemmatize(token) for token in tokens]

def LemNormalize(text):
    return LemTokens(nltk.word_tokenize(text.lower().translate(remove_punct_dict)))

In [9]:
from sklearn.feature_extraction.text import TfidfVectorizer

tfidf_vect = TfidfVectorizer(tokenizer=LemNormalize, stop_words='english', ngram_range=(1, 2), min_df=0.05, max_df=0.85)

feature_vect = tfidf_vect.fit_transform(document_df['opinion_text'])

/usr/local/anaconda3/lib/python3.9/site-packages/sklearn/feature_extraction/text.py:396: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['ha', 'le', 'u', 'wa'] not in stop_words.
  warnings.warn(


In [10]:
from sklearn.cluster import KMeans

km_cluster = KMeans(n_clusters=5, max_iter=10000, random_state=0)
km_cluster.fit(feature_vect)
cluster_label = km_cluster.labels_
cluster_centers = km_cluster.cluster_centers_
document_df['cluster_label'] = cluster_label
document_df.head()

,filename,opinion_text,cluster_label
0,battery-life_ipod_nano_8gb,...,3
1,gas_mileage_toyota_camry_2007,...,2
2,room_holiday_inn_london,...,0
3,location_holiday_inn_london,...,0
4,staff_bestwestern_hotel_sfo,...,0


In [11]:
document_df[document_df['cluster_label'] == 0].sort_values(by='filename')

,filename,opinion_text,cluster_label
31,bathroom_bestwestern_hotel_sfo,...,0
17,food_holiday_inn_london,...,0
32,food_swissotel_chicago,...,0
49,free_bestwestern_hotel_sfo,...,0
39,location_bestwestern_hotel_sfo,...,0
3,location_holiday_inn_london,...,0
50,parking_bestwestern_hotel_sfo,...,0
28,price_holiday_inn_london,...,0
2,room_holiday_inn_london,...,0
46,rooms_bestwestern_hotel_sfo,...,0


In [13]:
document_df[document_df['cluster_label'] == 1].sort_values(by='filename')

,filename,opinion_text,cluster_label
33,accuracy_garmin_nuvi_255W_gps,...,1
34,directions_garmin_nuvi_255W_gps,...,1
48,display_garmin_nuvi_255W_gps,...,1
21,features_windows7,...,1
12,keyboard_netbook_1005ha,...,1
10,satellite_garmin_nuvi_255W_gps,...,1
8,screen_garmin_nuvi_255W_gps,...,1
25,screen_ipod_nano_8gb,...,1
37,screen_netbook_1005ha,...,1
7,size_asus_netbook_1005ha,...,1


In [14]:
document_df[document_df['cluster_label'] == 2].sort_values(by='filename')

,filename,opinion_text,cluster_label
18,comfort_honda_accord_2008,...,2
43,comfort_toyota_camry_2007,...,2
1,gas_mileage_toyota_camry_2007,...,2
45,interior_honda_accord_2008,...,2
22,interior_toyota_camry_2007,...,2
35,mileage_honda_accord_2008,...,2
47,performance_honda_accord_2008,...,2
42,quality_toyota_camry_2007,...,2
29,seats_honda_accord_2008,...,2
23,transmission_toyota_camry_2007,...,2


In [15]:
document_df[document_df['cluster_label'] == 3].sort_values(by='filename')

,filename,opinion_text,cluster_label
9,battery-life_amazon_kindle,...,3
0,battery-life_ipod_nano_8gb,...,3
11,battery-life_netbook_1005ha,...,3
15,performance_netbook_1005ha,...,3
24,sound_ipod_nano_8gb,headphone jack i got a clear case for it a...,3


In [17]:
document_df[document_df['cluster_label'] == 4].sort_values(by='filename')

,filename,opinion_text,cluster_label
26,buttons_amazon_kindle,...,4
36,eyesight-issues_amazon_kindle,...,4
44,fonts_amazon_kindle,...,4
38,navigation_amazon_kindle,...,4
41,price_amazon_kindle,...,4


In [18]:
km_cluster = KMeans(n_clusters=3, max_iter=10000, random_state=0)
km_cluster.fit(feature_vect)
cluster_label = km_cluster.labels_
cluster_centers = km_cluster.cluster_centers_

document_df['cluster_label'] = cluster_label
document_df.sort_values(by='cluster_label')

,filename,opinion_text,cluster_label
0,battery-life_ipod_nano_8gb,...,0
48,display_garmin_nuvi_255W_gps,...,0
44,fonts_amazon_kindle,...,0
41,price_amazon_kindle,...,0
40,speed_windows7,...,0
38,navigation_amazon_kindle,...,0
37,screen_netbook_1005ha,...,0
36,eyesight-issues_amazon_kindle,...,0
34,directions_garmin_nuvi_255W_gps,...,0
33,accuracy_garmin_nuvi_255W_gps,...,0


In [20]:
print('cluster_centers.shape: ', cluster_centers.shape)
print(cluster_centers)

cluster_centers.shape:  (3, 4611)
[[0.01005322 0.         0.         ... 0.00706287 0.         0.        ]
 [0.         0.00092551 0.         ... 0.         0.         0.        ]
 [0.         0.00099499 0.00174637 ... 0.         0.00183397 0.00144581]]


In [21]:
def get_cluster_details(cluster_model, cluster_data, feature_names, clusters_num, top_n_features=10):
    cluster_details = {}

    centroid_feature_orderd_index = cluster_model.cluster_centers_.argsort()[:, ::-1]

    for cluster_num in range(clusters_num):
        cluster_details[cluster_num] = {}
        cluster_details[cluster_num]['cluster'] = cluster_num

        top_features_indexes = centroid_feature_orderd_index[cluster_num, :top_n_features]
        top_features = [ feature_names[index] for index in top_features_indexes ]

        top_features_values = cluster_model.cluster_centers_[cluster_num, top_features_indexes].tolist()

        cluster_details[cluster_num]['top_features'] = top_features
        cluster_details[cluster_num]['top_features_value'] = top_features_values
        filenames = cluster_data[cluster_data['cluster_label'] == cluster_num]['filename']
        filenames = filenames.values.tolist()

        cluster_details[cluster_num]['filenames'] = filenames

    return cluster_details

In [22]:
def print_cluster_details(cluster_details):
    for cluster_num, cluster_detail in cluster_details.items():
        print('cluster_num: ', cluster_num)
        print('cluster_detail[\'top_features\']: ', cluster_detail['top_features'])
        print(cluster_detail['filenames'][:7])

In [24]:
feature_names = tfidf_vect.get_feature_names()

cluster_details = get_cluster_details(cluster_model=km_cluster, cluster_data=document_df, feature_names=feature_names, clusters_num=3, top_n_features=10 )
print_cluster_details(cluster_details)

cluster_num:  0
cluster_detail['top_features']:  ['screen', 'battery', 'keyboard', 'battery life', 'life', 'kindle', 'direction', 'video', 'size', 'voice']
['battery-life_ipod_nano_8gb', 'voice_garmin_nuvi_255W_gps', 'speed_garmin_nuvi_255W_gps', 'size_asus_netbook_1005ha', 'screen_garmin_nuvi_255W_gps', 'battery-life_amazon_kindle', 'satellite_garmin_nuvi_255W_gps']
cluster_num:  1
cluster_detail['top_features']:  ['interior', 'seat', 'mileage', 'comfortable', 'gas', 'gas mileage', 'transmission', 'car', 'performance', 'quality']
['gas_mileage_toyota_camry_2007', 'comfort_honda_accord_2008', 'interior_toyota_camry_2007', 'transmission_toyota_camry_2007', 'seats_honda_accord_2008', 'mileage_honda_accord_2008', 'quality_toyota_camry_2007']
cluster_num:  2
cluster_detail['top_features']:  ['room', 'hotel', 'service', 'staff', 'food', 'location', 'bathroom', 'clean', 'price', 'parking']
['room_holiday_inn_london', 'location_holiday_inn_london', 'staff_bestwestern_hotel_sfo', 'service_swis